In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [82]:
class TeamA_Predictor():
    # stocks is a list of stocks to look up
    # investment is starting dollar amount
    def __init__(self):
        self.training_tickers = ["TMO","AMZN","AAPL","GM","IBM","TWTR","CBS","TM","BIO"]
        self.train_data = pd.concat([self.get_data(stock) for stock in self.training_tickers])
        self.X, self.y = self.set_vars(self.train_data)
        self.model = LinearRegression().fit(self.X, self.y)
        
    def get_data(self, stock):
        url = (
            'http://www.google.com/finance/historical?output=csv&startdate=2000-01-01&enddate=2017-01-01&q={}'.format(stock)
        )
        df = pd.read_csv(url, encoding = 'utf8')
        df = df.drop(['Open', 'High', 'Low', 'Volume'], axis = 1)
        df.set_index(u'﻿Date', inplace = True)
        df.index = df.index.to_datetime()
        
        offset_1, offset_2, offset_3 = 252, (252*2), (252*3)
        df_length = len(df) - offset_3
        x_3 = df[:df_length] # target variable
        x_2 = df[offset_1:df_length + offset_1] # one year offset
        x_1 = df[offset_2:df_length + offset_2] # two year offset
        x_0 = df[offset_3: df_length + offset_3]
        
        x_3 = x_3.reset_index().drop('index', axis=1)
        x_2 = x_2.reset_index().drop('index', axis=1)
        x_1 = x_1.reset_index().drop('index', axis=1)
        x_0 = x_0.reset_index().drop('index', axis=1)
        
        frames = [x_0, x_1, x_2, x_3]
        
        final_df = pd.concat(frames, axis=1)
        final_df.columns = ['x_0', 'x_1', 'x_2', 'y']
        return final_df
        
    def set_vars(self, df):
    
        X = df[['x_0','x_1','x_2']]
        y = df.y
        
        return X, y
    
    def predict2014_to_2017(self, stocks, investment):
        test_data = pd.concat([self.get_data(stock) for stock in stocks])
        test_X, test_y = self.set_vars(test_data)
        pred_y = self.model.predict(test_X)
        return pred_y
   

In [83]:
pr = TeamA_Predictor()

In [85]:
predictions = pr.predict2014_to_2017(["VIAB","GOOG","F","MSFT"], 10000)

In [86]:
len(predictions)

10876